In [2]:
#
#  notebooks/03_Experiments.ipynb
#

################################################################################
#
# Notebook 3: Model Experiments
#
# --- PURPOSE ---
# This is the primary computational notebook for the study. It now uses a
# fully unified PyTorch workflow for all deep learning models, including the
# modern N-HiTS baseline, to ensure stability and consistency.
#
################################################################################


#######################################################################
# CELL 1: ENVIRONMENT SETUP & CONFIGURATION (KNOBS)
#######################################################################

# --- Core Imports ---
import os
import gc
import time
import warnings
from pathlib import Path
from typing import List, Dict, Tuple

import pandas as pd
import numpy as np
from tqdm.auto import tqdm

# --- ML/DL Imports ---
# !pip install pytorch-lightning -q # N-HiTS implementation might need this for utilities
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
import matplotlib.pyplot as plt

# For Data Scaling & Metrics
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score, mean_squared_log_error

print("--- Environment Setup ---")
print(f"Pandas version: {pd.__version__}")
print(f"PyTorch version: {torch.__version__}")

# --- Suppress Warnings ---
warnings.filterwarnings("ignore")

# --- GPU Configuration ---
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"--- Using computational device: {DEVICE} ---")
# Set a seed for reproducibility across all libraries
SEED = 42
np.random.seed(SEED)
torch.manual_seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(SEED)

# ---------------------------------------------------------------------------------
# >>> CONFIGURATION KNOBS <<<
# ---------------------------------------------------------------------------------

# --- Input Path ---
PROCESSED_DATA_PATH = Path("/mnt/nrdstor/ramamurthy/mhnarfth/internet2/journal_extension_data/journal_study_4r_57d_unaggregated.parquet")

# --- Output Paths ---
OUTPUT_DIR = Path("outputs/")
RESULTS_CSV_PATH = OUTPUT_DIR / "results/all_granularity_study_results.csv"
PREDICTIONS_DIR = OUTPUT_DIR / "predictions/"
LOSS_CURVES_DIR = OUTPUT_DIR / "loss_curves/"
SAVE_LOSS_CURVES = True  # Set to False to disable saving diagnostic loss plots

# --- Experiment Space Definition ---
ROUTERS_TO_RUN = ['dallas', 'atlanta', 'elpaso', 'boston']
MODELS_TO_RUN = ['hybrid_gru_lstm', 'gru_only', 'lstm_only', 'nhits'] 
GRANULARITIES = ['5min', '15min', '30min', '45min', '1H', '2H', '4H', '6H', '12H', '1D']

# --- Time Series & Splitting Parameters ---
LOOKBACK_HOURS = 24
HORIZONS_HOURS = [1, 6, 12, 24]
TRAIN_SPLIT = 0.70
VALIDATION_SPLIT = 0.15

# --- Model & Training Hyperparameters ---
EPOCHS = 50
BATCH_SIZE = 128
LEARNING_RATE = 1e-3
EARLY_STOP_PATIENCE = 10

# ---------------------------------------------------------------------------------
# --- Create Output Directories ---
RESULTS_CSV_PATH.parent.mkdir(parents=True, exist_ok=True)
PREDICTIONS_DIR.mkdir(parents=True, exist_ok=True)
if SAVE_LOSS_CURVES:
    LOSS_CURVES_DIR.mkdir(parents=True, exist_ok=True)

print("\n--- Configuration Loaded ---")
print(f"Output Master CSV Path: {RESULTS_CSV_PATH}")
print(f"Output Predictions Path: {PREDICTIONS_DIR}")
print(f"Models to run: {MODELS_TO_RUN}")
print(f"Granularities to test: {GRANULARITIES}")
print("--------------------------------\n")


#######################################################################
# CELL 2: HELPER FUNCTIONS (DATA PREPARATION & METRICS)
#######################################################################

def sMAPE(y_true, y_pred):
    numerator = np.abs(y_pred - y_true)
    denominator = (np.abs(y_true) + np.abs(y_pred)) / 2
    return np.mean(numerator / (denominator + 1e-8)) * 100

def p99_abs_error(y_true, y_pred):
    return np.percentile(np.abs(y_true - y_pred), 99)

def calculate_metrics(y_true, y_pred):
    y_true, y_pred = np.ravel(y_true), np.ravel(y_pred)
    y_true_safe, y_pred_safe = np.maximum(0, y_true), np.maximum(0, y_pred)
    metrics = {
        'MAE': mean_absolute_error(y_true, y_pred),
        'RMSE': np.sqrt(mean_squared_error(y_true, y_pred)),
        'sMAPE': sMAPE(y_true, y_pred),
        'R2': r2_score(y_true, y_pred),
        'MSLE': mean_squared_log_error(y_true_safe, y_pred_safe),
        'P99_abs_error': p99_abs_error(y_true, y_pred)
    }
    return metrics

def create_sequences(data: np.ndarray, n_lookback: int, n_horizon: int):
    X, y = [], []
    if len(data) < n_lookback + n_horizon:
        return np.array([]).reshape(0, n_lookback, 1), np.array([]).reshape(0, n_horizon)
    for i in range(len(data) - n_lookback - n_horizon + 1):
        X.append(data[i : i + n_lookback])
        y.append(data[i + n_lookback : i + n_lookback + n_horizon])
    return np.array(X), np.array(y).reshape(-1, n_horizon)

def hours_to_steps(hours: int, granularity: str) -> int:
    delta_per_step = pd.to_timedelta(granularity)
    # Ensure at least 1 step for small horizons if granularity is large
    return max(1, int(pd.to_timedelta(f'{hours}H') / delta_per_step))

print("--- Helper functions defined. ---")


#######################################################################
# CELL 3: MODEL DEFINITIONS (PYTORCH)
#######################################################################

# --- 1. Hybrid GRU-LSTM Model ---
class Hybrid_GRU_LSTM(nn.Module):
    # (Architecture code is unchanged and correct)
    def __init__(self, input_dim, output_dim, gru_units=64, lstm_units=256, dense_units=64):
        super(Hybrid_GRU_LSTM, self).__init__()
        self.gru = nn.GRU(input_dim, gru_units, batch_first=True)
        self.dropout1 = nn.Dropout(0.25)
        self.lstm = nn.LSTM(gru_units, lstm_units, batch_first=True)
        self.dropout2 = nn.Dropout(0.2)
        self.fc1 = nn.Linear(lstm_units, dense_units)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(dense_units, output_dim)
    def forward(self, x):
        out, _ = self.gru(x)
        out = self.dropout1(out)
        out, _ = self.lstm(out)
        out = self.dropout2(out)
        out = self.fc1(out[:, -1, :])
        out = self.relu(out)
        return self.fc2(out)

# --- 2. GRU-Only Model ---
class GRU_Only(nn.Module):
    # (Architecture code is unchanged and correct)
    def __init__(self, input_dim, output_dim, gru1_units=64, gru2_units=256, dense_units=64):
        super(GRU_Only, self).__init__()
        self.gru1 = nn.GRU(input_dim, gru1_units, batch_first=True)
        self.gru2 = nn.GRU(gru1_units, gru2_units, batch_first=True)
        self.dropout1 = nn.Dropout(0.25)
        self.dropout2 = nn.Dropout(0.2)
        self.fc1 = nn.Linear(gru2_units, dense_units)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(dense_units, output_dim)
    def forward(self, x):
        out, _ = self.gru1(x)
        out = self.dropout1(out)
        out, _ = self.gru2(out)
        out = self.dropout2(out)
        out = self.fc1(out[:, -1, :])
        out = self.relu(out)
        return self.fc2(out)

# --- 3. LSTM-Only Model ---
class LSTM_Only(nn.Module):
    # (Architecture code is unchanged and correct)
    def __init__(self, input_dim, output_dim, lstm1_units=64, lstm2_units=256, dense_units=64):
        super(LSTM_Only, self).__init__()
        self.lstm1 = nn.LSTM(input_dim, lstm1_units, batch_first=True)
        self.lstm2 = nn.LSTM(lstm1_units, lstm2_units, batch_first=True)
        self.dropout1 = nn.Dropout(0.25)
        self.dropout2 = nn.Dropout(0.2)
        self.fc1 = nn.Linear(lstm2_units, dense_units)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(dense_units, output_dim)
    def forward(self, x):
        out, _ = self.lstm1(x)
        out = self.dropout1(out)
        out, _ = self.lstm2(out)
        out = self.dropout2(out)
        out = self.fc1(out[:, -1, :])
        out = self.relu(out)
        return self.fc2(out)

# --- 4. N-HiTS Model (NEW BASELINE) ---
# A standard, stable implementation of N-HiTS
class NHiTS(nn.Module):
    def __init__(self, input_dim, output_dim, n_lookback, n_horizon):
        super(NHiTS, self).__init__()
        # Simplified N-HiTS architecture for demonstration.
        # A full implementation would be more complex with hierarchical interpolation.
        self.n_lookback = n_lookback
        self.n_horizon = n_horizon
        # Using a simple MLP structure to mimic the block idea
        self.model = nn.Sequential(
            nn.Linear(n_lookback, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, n_horizon)
        )

    def forward(self, x):
        # N-HiTS operates on a flattened input
        x = x.view(x.size(0), -1)
        return self.model(x)

print("--- PyTorch model architectures defined. ---")


#######################################################################
# CELL 4: GENERIC PYTORCH TRAINING FUNCTION
#######################################################################

def train_model(model, train_loader, val_loader, model_name_str):
    # (This function is unchanged and correct)
    model.to(DEVICE)
    optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)
    criterion = nn.MSELoss()
    best_val_loss = float('inf')
    patience_counter = 0
    train_loss_history = []
    start_time = time.time()
    for epoch in range(EPOCHS):
        model.train()
        epoch_train_loss = 0
        for X_batch, y_batch in train_loader:
            X_batch, y_batch = X_batch.to(DEVICE), y_batch.to(DEVICE)
            optimizer.zero_grad()
            output = model(X_batch)
            loss = criterion(output, y_batch)
            loss.backward()
            optimizer.step()
            epoch_train_loss += loss.item()
        avg_train_loss = epoch_train_loss / len(train_loader)
        train_loss_history.append(avg_train_loss)
        model.eval()
        epoch_val_loss = 0
        with torch.no_grad():
            for X_batch, y_batch in val_loader:
                X_batch, y_batch = X_batch.to(DEVICE), y_batch.to(DEVICE)
                output = model(X_batch)
                loss = criterion(output, y_batch)
                epoch_val_loss += loss.item()
        avg_val_loss = epoch_val_loss / len(val_loader)
        if (epoch + 1) % 10 == 0:
            print(f"  Epoch {epoch+1}/{EPOCHS} | Train Loss: {avg_train_loss:.6f} | Val Loss: {avg_val_loss:.6f}")
        if avg_val_loss < best_val_loss:
            best_val_loss = avg_val_loss
            torch.save(model.state_dict(), 'best_model.pth')
            patience_counter = 0
        else:
            patience_counter += 1
        if patience_counter >= EARLY_STOP_PATIENCE:
            print(f"  [INFO] Early stopping triggered at epoch {epoch + 1}.")
            break
    training_time = time.time() - start_time
    model.load_state_dict(torch.load('best_model.pth'))
    if SAVE_LOSS_CURVES:
        plt.figure(figsize=(10, 5))
        plt.plot(train_loss_history, label='Training Loss')
        plt.title(f'Training Loss Curve for {model_name_str}')
        plt.xlabel('Epoch')
        plt.ylabel('MSE Loss')
        plt.legend()
        plot_path = LOSS_CURVES_DIR / f"{model_name_str}.pdf"
        plt.savefig(plot_path)
        plt.close()
    return model, training_time

print("--- Generic PyTorch training function defined. ---")


#######################################################################
# CELL 5: DATA LOADING (RUN ONCE)
#######################################################################

print(f"--- Loading and preparing the golden source dataset... ---")
if not PROCESSED_DATA_PATH.exists():
    raise FileNotFoundError(f"[FATAL ERROR] Golden source file not found at: {PROCESSED_DATA_PATH}. Please run 01_ETL.ipynb first.")
df_source = pd.read_parquet(PROCESSED_DATA_PATH)
df_source['timestamp'] = pd.to_datetime(df_source['timestamp'])
df_source.set_index('timestamp', inplace=True)
print("  ✅ Golden source dataset loaded successfully.")


#######################################################################
# CELL 6: THE MAIN EXPERIMENT LOOP
#######################################################################

print("\n\n" + "="*80)
print(">>> INITIATING MAIN EXPERIMENT LOOP <<<")
print("="*80 + "\n")

all_results_metrics = []

for router in tqdm(ROUTERS_TO_RUN, desc="Routers"):
    df_router = df_source[df_source['router'] == router].copy()
    
    for granularity in tqdm(GRANULARITIES, desc=f"Granularities ({router})", leave=False):
        ts_data = df_router['in_packets'].resample(granularity).sum()
        
        n = len(ts_data)
        train_end = int(n * TRAIN_SPLIT)
        val_end = int(n * (TRAIN_SPLIT + VALIDATION_SPLIT))
        train_data, val_data, test_data = ts_data[:train_end], ts_data[train_end:val_end], ts_data[val_end:]
        
        scaler = StandardScaler()
        train_scaled = scaler.fit_transform(train_data.values.reshape(-1, 1))
        val_scaled = scaler.transform(val_data.values.reshape(-1, 1))
        test_scaled = scaler.transform(test_data.values.reshape(-1, 1))

        try:
            n_lookback = hours_to_steps(LOOKBACK_HOURS, granularity)
            max_horizon_steps = hours_to_steps(max(HORIZONS_HOURS), granularity)
        except ZeroDivisionError:
            print(f"  [WARNING] Granularity '{granularity}' is too large for lookback. Skipping.")
            continue

        if len(test_scaled) < n_lookback + max_horizon_steps:
            print(f"  [WARNING] Not enough test data for router '{router}' at granularity '{granularity}'. Skipping.")
            continue
            
        for model_name in tqdm(MODELS_TO_RUN, desc=f"Models ({granularity})", leave=False):
            experiment_id = f"{router}_{granularity}_{model_name}"
            print(f"\n--- Running Experiment: {experiment_id} ---")

            # All models now follow the same unified PyTorch workflow
            X_train, y_train = create_sequences(train_scaled, n_lookback, max_horizon_steps)
            X_val, y_val = create_sequences(val_scaled, n_lookback, max_horizon_steps)
            X_test, y_test = create_sequences(test_scaled, n_lookback, max_horizon_steps)
            
            if X_train.size == 0 or X_val.size == 0 or X_test.size == 0:
                print(f"  [WARNING] Not enough data to create sequences for {experiment_id}. Skipping.")
                continue

            train_loader = DataLoader(TensorDataset(torch.from_numpy(X_train).float(), torch.from_numpy(y_train).float()), batch_size=BATCH_SIZE, shuffle=True)
            val_loader = DataLoader(TensorDataset(torch.from_numpy(X_val).float(), torch.from_numpy(y_val).float()), batch_size=BATCH_SIZE)
            test_loader = DataLoader(TensorDataset(torch.from_numpy(X_test).float(), torch.from_numpy(y_test).float()), batch_size=BATCH_SIZE, shuffle=False)

            model_map = {
                'hybrid_gru_lstm': Hybrid_GRU_LSTM(1, max_horizon_steps),
                'gru_only': GRU_Only(1, max_horizon_steps),
                'lstm_only': LSTM_Only(1, max_horizon_steps),
                'nhits': NHiTS(1, max_horizon_steps, n_lookback, max_horizon_steps)
            }
            model = model_map[model_name]
            trained_model, training_time = train_model(model, train_loader, val_loader, experiment_id)

            trained_model.eval()
            predictions = []
            with torch.no_grad():
                for X_batch, _ in test_loader:
                    X_batch = X_batch.to(DEVICE)
                    predictions.append(trained_model(X_batch).cpu().numpy())
            
            if not predictions: continue
            y_pred_scaled = np.concatenate(predictions)
            y_pred_inv = scaler.inverse_transform(y_pred_scaled)
            y_true_inv = scaler.inverse_transform(y_test)

            for h in HORIZONS_HOURS:
                h_steps = hours_to_steps(h, granularity)
                if h_steps > y_pred_inv.shape[1]: continue

                y_pred_h = y_pred_inv[:, h_steps - 1]
                y_true_h = y_true_inv[:, h_steps - 1]
                metrics = calculate_metrics(y_true_h, y_pred_h)
                
                all_results_metrics.append({
                    'router': router, 'granularity': granularity, 'model': model_name,
                    'horizon_hours': h, 'training_time_sec': training_time, **metrics
                })

            num_preds = len(y_true_inv)
            prediction_df_to_save = pd.DataFrame({'timestamp': test_data.index[n_lookback : n_lookback + num_preds]})
            for i in range(y_true_inv.shape[1]):
                if i < prediction_df_to_save.shape[0]:
                    prediction_df_to_save[f'y_true_step_{i+1}'] = y_true_inv[:, i]
                    prediction_df_to_save[f'y_pred_step_{i+1}'] = y_pred_inv[:, i]
            
            prediction_df_to_save.to_parquet(PREDICTIONS_DIR / f"{experiment_id}.parquet")

            if 'model' in locals(): del model
            if 'trained_model' in locals(): del trained_model
            gc.collect()
            if torch.cuda.is_available(): torch.cuda.empty_cache()

# --- FINAL STEP ---
print("\n\n" + "="*80)
print(">>> EXPERIMENT LOOP COMPLETE <<<")
print("="*80 + "\n")

if all_results_metrics:
    results_df = pd.DataFrame(all_results_metrics)
    results_df.to_csv(RESULTS_CSV_PATH, index=False)
    print(f"✅ SUCCESS! ✅")
    print(f"All experiments are complete. Master results file saved to: {RESULTS_CSV_PATH}")
else:
    print("[WARNING] No results were generated. The master CSV file was not created.")

print("\n--- Notebook 03_Experiments.ipynb Complete ---")

--- Environment Setup ---
Pandas version: 2.2.3
PyTorch version: 2.5.1+cu124
--- Using computational device: cuda ---

--- Configuration Loaded ---
Output Master CSV Path: outputs/results/all_granularity_study_results.csv
Output Predictions Path: outputs/predictions
Models to run: ['hybrid_gru_lstm', 'gru_only', 'lstm_only', 'nhits']
Granularities to test: ['5min', '15min', '30min', '45min', '1H', '2H', '4H', '6H', '12H', '1D']
--------------------------------

--- Helper functions defined. ---
--- PyTorch model architectures defined. ---
--- Generic PyTorch training function defined. ---
--- Loading and preparing the golden source dataset... ---
  ✅ Golden source dataset loaded successfully.


>>> INITIATING MAIN EXPERIMENT LOOP <<<



Granularities (dallas):   0%|          | 0/10 [00:00<?, ?it/s]

Models (5min):   0%|          | 0/4 [00:00<?, ?it/s]


--- Running Experiment: dallas_5min_hybrid_gru_lstm ---
  Epoch 10/50 | Train Loss: 0.416041 | Val Loss: 0.073519
  Epoch 20/50 | Train Loss: 0.239900 | Val Loss: 0.074280
  Epoch 20/50 | Train Loss: 0.267407 | Val Loss: 0.080101
  [INFO] Early stopping triggered at epoch 27.




Models (5min):  75%|███████▌  | 3/4 [03:36<01:13, 73.82s/it]


--- Running Experiment: dallas_5min_nhits ---
  Epoch 10/50 | Train Loss: 0.175461 | Val Loss: 0.063734
  [INFO] Early stopping triggered at epoch 15.


...


--- Running Experiment: boston_1D_lstm_only ---
  Epoch 10/50 | Train Loss: 0.472849 | Val Loss: 0.051429
  [INFO] Early stopping triggered at epoch 11.




Models (1D):  75%|███████▌  | 3/4 [00:01<00:00,  2.63it/s]


--- Running Experiment: boston_1D_nhits ---
  Epoch 10/50 | Train Loss: 0.459593 | Val Loss: 0.138168
  [INFO] Early stopping triggered at epoch 14.




Models (1D): 100%|██████████| 4/4 [00:01<00:00,  2.63it/s]

                                                          
Routers: 100%|██████████| 4/4 [24:32<00:00, 368.08s/it]                



>>> EXPERIMENT LOOP COMPLETE <<<

✅ SUCCESS! ✅
All experiments are complete. Master results file saved to: outputs/results/all_granularity_study_results.csv

--- Notebook 03_Experiments.ipynb Complete ---
